<a href="https://colab.research.google.com/github/RLWH/reinforcement-learning-notebook/blob/master/Ch3%20MPD/Ch3_Gridworld_MDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Gridworld problem and solving gridworld by Dynamic Programming
1. Policy Evaluation
2. Policy Iteration
3. Value Iteration

## Setup OpenAI Rendering in Colab

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only

import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

In [3]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

In [0]:
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
env = wrap_env(gym.make('CartPole-v0'))
# env.reset()
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
#         print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

Episode finished after 12 timesteps
Episode finished after 13 timesteps
Episode finished after 14 timesteps
Episode finished after 21 timesteps
Episode finished after 23 timesteps
Episode finished after 12 timesteps
Episode finished after 16 timesteps
Episode finished after 9 timesteps
Episode finished after 12 timesteps
Episode finished after 15 timesteps
Episode finished after 24 timesteps
Episode finished after 17 timesteps
Episode finished after 11 timesteps
Episode finished after 18 timesteps
Episode finished after 28 timesteps
Episode finished after 17 timesteps
Episode finished after 31 timesteps
Episode finished after 33 timesteps
Episode finished after 12 timesteps
Episode finished after 14 timesteps


In [0]:
show_video()

## Setup the Gridworld Environment
https://github.com/dennybritz/reinforcement-learning/blob/master/lib/envs/gridworld.py

In [0]:
import numpy as np
import sys

from collections import defaultdict
from gym.envs.toy_text import discrete

In [0]:
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

In [0]:
class GridworldEnv(discrete.DiscreteEnv):
  """
  The Gridworld environment that is a discrete space
  
  Grid World environment from Sutton's Reinforcement Learning book chapter 4.
  You are an agent on an MxN grid and your goal is to reach the terminal
  state at the top left or the bottom right corner.

  For example, a 4x4 grid looks as follows:

  T  o  o  o
  o  x  o  o
  o  o  o  o
  o  o  o  T

  x is your position and T are the two terminal states.

  You can take actions in each direction (UP=0, RIGHT=1, DOWN=2, LEFT=3).
  Actions going off the edge leave you in your current state.
  You receive a reward of -1 at each step until you reach a terminal state.
  
  A toy text discrete environment has the followings:
  1. nS: Number of states
  2. nA: Number of actions
  3. P: transitions (*)
  4. isd: initial state distribution
  
  (*) dictionary dict of dicts of lists, where
    P[s][a] == [(probability, nextstate, reward, done), ...]
  (**) list or array of length nS

  """
  
  metadata = {'render.modes': ['human', 'ansi']}
  
  def __init__(self, shape=[4, 4]):
    if not isinstance(shape, (list, tuple)) or not len(shape) == 2:
      raise ValueError('shape argument must be a list/tuple of length 2')
      
    self.shape = shape
    
    # The grid world has n x n states
    nS = np.prod(shape)
    
    # There are only 4 possible actions
    actions = [UP, RIGHT, DOWN, LEFT]
    nA = len(actions)
    
    # Define the maximum board shape
    MAX_Y = shape[0]
    MAX_X = shape[1]
    
    # Define terminate state
    is_done = lambda s: s == 0 or s == (nS - 1)
    
    
    # Initialise P
    P = {}
    
    grid = np.arange(nS).reshape(shape)
    it = np.nditer(grid, flags=['multi_index'])
    
    while not it.finished:
      s = it.iterindex
      y, x = it.multi_index
      
      # Define Reward is -1 for each step
      reward = 0.0 if is_done(s) else -1.0
      
      # Initialize P
      P[s] = {a: [] for a in actions}
      
      if is_done(s):
        for a in actions:
          P[s][a] = [(1.0, s, reward, True)]
      else:
        
        # Define the next state of each state
        ns_up = s if y == 0 else s - MAX_X
        ns_right = s if x == (MAX_X - 1) else s + 1
        ns_down = s if y == (MAX_Y - 1) else s + MAX_X
        ns_left = s if x == 0 else s - 1
        
        P[s][UP] = [(1.0, ns_up, reward, is_done(ns_up))]
        P[s][RIGHT] = [(1.0, ns_right, reward, is_done(ns_right))]
        P[s][DOWN] = [(1.0, ns_down, reward, is_done(ns_down))]
        P[s][LEFT] = [(1.0, ns_left, reward, is_done(ns_left))]
        
        
      it.iternext()
      
      # Define Initial state distribution is uniform
      isd = np.ones(nS) / nS
      
      self.P = P
      
      
      super().__init__(nS, nA, P, isd)
    
    
  def render(self, mode='human', close=False):
    """
    Render the environment
    """
    if close:
      return
    
    outfile = StringIO() if mode == 'ansi' else sys.stdout
    
    grid = np.arange(self.nS).reshape(self.shape)
    it = np.nditer(grid, flags=['multi_index'])
    
    while not it.finished:
      s = it.iterindex
      y, x = it.multi_index
      
      if self.s == s:
        output = "x "
      elif s == 0 or s == self.nS - 1:
        output = "T "
      else:
        output = "o "
        
      if x == 0:
        output = output.lstrip()
        
      if x == self.shape[1] - 1:
        output = output.rstrip()

      outfile.write(output)
      
      if x == self.shape[1] - 1:
        outfile.write("\n")
        
      it.iternext()
    
    

In [10]:
env = GridworldEnv()
for t in range(100):
    env.render()
#         print(observation)
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done:
        print("Episode finished after {} timesteps".format(t+1))
        break
env.close()

T o o o
o x o o
o o o o
o o o T
T o o o
x o o o
o o o o
o o o T
T o o o
o x o o
o o o o
o o o T
T x o o
o o o o
o o o o
o o o T
Episode finished after 4 timesteps


In [11]:
env.P[5]

{0: [(1.0, 1, -1.0, False)],
 1: [(1.0, 6, -1.0, False)],
 2: [(1.0, 9, -1.0, False)],
 3: [(1.0, 4, -1.0, False)]}

## Solving Gridworld by dynamic programming
We will solve a Bellman equation using two algorithms:
1. Value iteration
2. Policy iteration

Q(s,a) = Transition probability * ( Reward probability + gamma * value_of_next_state)

### Value Iteration
In value iteration, we start off with a random value function, then look for a new improved value function in iterative fashion until we find the optimal value function.

1. Initialise random value function
2. For each state, calculate Q(s, a)
3. Since V(s) = Max W(s, a), update the value function with max value of Q(s, a)
4. If V(S) is optimal,  then stop. Repeat otherwise.

#### Environment Inspection

In [12]:
print(env.observation_space.n)

16


In [13]:
print(env.action_space.n)

4


#### Initialise the value table

In [0]:
value_table = np.zeros(env.observation_space.n)
no_of_iterations = 100

In [16]:
value_table.reshape(4, 4)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

#### For each state, calculate Q

In [0]:
# Upon starting each iteration, we copy the value_table to updated_value_table
for i in range(no_of_iterations):
  updated_value_table = np.copy(value_table)

In [0]:
for state in range(env.observation_space.n):
  Q_value = []
  
  for action in range(env.action_space.n):
    next_states_rewards = []
    for next_sr in env.P[state][action]:
      trans_prob, next_state, reward_prob, _ = next_sr
      next_states_rewards.append((trans_prob * (reward_prob + 1 * updated_value_table[next_state])))
      Q_value.append(np.sum(next_states_rewards))
      
      # Pick up the maximum Q value and update it as value of a state
      value_table[state] = max(Q_value)

In [0]:
value_table.reshape((4, 4))

array([[ 0., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1.,  0.]])

#### For each state, calculate Q - Combine them into a function

In [0]:
def value_iteration(env, gamma=1.0, no_of_iterations=100000):
  """
  Value iteration function
  """
  
  # Initialise value tables
  value_table = np.zeros(env.observation_space.n)
  threshold = 1e-20
  
  print(no_of_iterations)
  
  # Table update
  for i in range(no_of_iterations):
    
    # Copy the current value table to updated value table
    updated_value_table = np.copy(value_table)
    
    for state in range(env.observation_space.n):
      print("state: %s" % state, end="")
      # For each of the state, calculate the state-action value q(s,a)
      Q_value = []
      
      for action in range(env.action_space.n):
        # One step ahead search for each action
        next_states_rewards = []
        
        for next_sr in env.P[state][action]:
          # P[s][a] == [(probability, nextstate, reward, done), ...]
#           print(next_sr)
          trans_prob, next_state, reward, _ = next_sr
          next_states_rewards.append((trans_prob * (reward + gamma * updated_value_table[next_state])))
          
        Q_value.append(np.sum(next_states_rewards))
          
      # Pick the maximum Q value and update it as value of a state
      value_table[state] = max(Q_value)
    
    print("Diff: %s" % np.sum(np.fabs(updated_value_table - value_table)))

    if (np.sum(np.fabs(updated_value_table - value_table))) <= threshold:
      print("Value-iteration converged at iteration %d" % (i + 1))
      break

  return value_table

In [0]:
env = GridworldEnv()

In [19]:
optimal_value_function = value_iteration(env=env, gamma=1.0)

100000
state: 0state: 1state: 2state: 3state: 4state: 5state: 6state: 7state: 8state: 9state: 10state: 11state: 12state: 13state: 14state: 15Diff: 14.0
state: 0state: 1state: 2state: 3state: 4state: 5state: 6state: 7state: 8state: 9state: 10state: 11state: 12state: 13state: 14state: 15Diff: 10.0
state: 0state: 1state: 2state: 3state: 4state: 5state: 6state: 7state: 8state: 9state: 10state: 11state: 12state: 13state: 14state: 15Diff: 4.0
state: 0state: 1state: 2state: 3state: 4state: 5state: 6state: 7state: 8state: 9state: 10state: 11state: 12state: 13state: 14state: 15Diff: 0.0
Value-iteration converged at iteration 4


In [20]:
optimal_value_function.reshape((4,4))

array([[ 0., -1., -2., -3.],
       [-1., -2., -3., -2.],
       [-2., -3., -2., -1.],
       [-3., -2., -1.,  0.]])

#### Extracting the optimal policy
After finding optimal value function, how can we extract the optimal policy from optimal function?  
We calculate the Q value using our optimal value action and pick up actions greedily for each state as the optimal policy.

We do this via a function called `extract_policy()`

#### Build a Q table for each state

A Q table looks like the following:

|State|Action|Value|
|--------|-----------|--------|
State 1|Action 1|Value 1|
State 1|Action 2|Value 2|
State 1|Action 3|Value 3|
State 1|Action 4|Value 4|



In [0]:
def extract_policy(value_table, gamma=1.0):
  
  # First, Define a random policy pi
  policy = np.zeros(env.observation_space.n)
  
  # Build a Q table - One step ahead
  for state in range(env.observation_space.n):
    # For each state, the Q table has num_actions 
    Q_table = np.zeros(env.action_space.n)

    for action in range(env.action_space.n):

      for next_sr in env.P[state][action]:
        # One step look ahead
        trans_prob, next_state, reward, _ = next_sr

        new_value = trans_prob * (reward + gamma * value_table[next_state])

        Q_table[action] = new_value
        
    policy[state] =  np.argmax(Q_table)
    
  return policy

In [0]:
optimal_policy = extract_policy(optimal_value_function)

In [24]:
optimal_policy.reshape(4,4)

array([[0., 3., 3., 2.],
       [0., 0., 0., 2.],
       [0., 0., 1., 2.],
       [0., 1., 1., 0.]])

### Policy Iteration
1. Policy evaluation: Evaluating the value function of a randomly estimated policy
2. Policy improvement: Upon evaluating the value function, if it is not optimal, we find a new improved policy $\pi'$

##### How can we evaluate the policies?  
We will evaluate our randomly initialized policies by computing value functions for them. If they are not good, then we find a new policy.   
We repeat this process until we find a good policy.

#### Steps
1. Initialize random policy $\pi$
2. Calculate value function V(S) for the policy
3. If V(S) is optimal -> End, otherwise find improved policy
4. Repeat

#### Initialize a random policy $\pi$
For each state, we get the action from policy, and compute the value function according to the `action` and `state` as folows

In [0]:
random_policy = np.ones([env.nS, env.nA]) / env.nA

In [28]:
random_policy[1]

array([0.25, 0.25, 0.25, 0.25])

#### Calculate the v(s) of the policy (Prediction problem) - In place version

Algorithm:
Input $\pi$, the policy to be evaluated  
Algorithm parameter: a small threshold $\theta > 0$ determining accuracy of estimation  
Initialize $V(s)$, for all $s \in S^+$, arbitrarily except that $V(terminal) = 0$

Loop:  
$\Delta \leftarrow 0$  
Loop for each $s \in S^+$:
* $v \leftarrow V(s)$
* $V(s) \leftarrow \sum_a \pi(a|s) \sum_{s', r}p(s', r|s, a)[r + \gamma V(s')]$
* $\Delta \leftarrow \text{max}(\Delta, |v - V(s|)$

Until $\Delta < \theta$

In [31]:
env.P[0][1]

[(1.0, 0, 0.0, True)]

In [0]:
def evaluate_policy(policy, gamma=1.0, theta=1e-10):
  
  value_table = np.zeros(env.nS)
  
  
  while True:
    # for each state in the environment, select the action according to the policy and compute the value table
    delta = 0

    for s in range(env.nS):

      v = value_table[s]
      new_v = 0

      for a, action_prob in enumerate(policy[s]):
        # For each action, look at the possible next states
        p, ns, r, _ = env.P[s][a][0]
        new_v += (action_prob * p * (r + gamma * value_table[ns]))

      # Update value table
      value_table[s] = new_v
          
      # How much of our value function has changed?
      delta = max(delta, np.abs(v - value_table[s]))
      
#     print(delta)

    if delta < theta:
      print("Policy value evaluated.")
      break
  
  return value_table

In [46]:
evaluate_policy(random_policy)

Policy value evaluated.


array([  0., -14., -20., -22., -14., -18., -20., -20., -20., -20., -18.,
       -14., -22., -20., -14.,   0.])

#### Policy Improvement

Suppose we have determined the value function $v_{\pi}$ for an arbitrary deterministic policy $\pi$.  
If at a step we would like to know whether or not we should change the policy to deterministically choose an action $a \neq \pi(s)$, and to know whether it will better off or worse off to change to the new policy, say $\pi'(s)$. How can we do this?

One way to compare whether a policy is better or worse off is to consider selecting action $a$ in state $s$, but following the policy $\pi$ thereafer. Hence, we can evaluate the action value of $a$

\begin{equation}
\begin{split}
q_\pi(s,a) & := \mathop{\mathbb{E}}[R_{t+1} + \gamma v_{\pi}(S_{t+1}) | S_t=s, A_t=a] \\
& = \sum_{s', r}p(s',r|s,a)[r + \gamma v_{\pi}(s')]
\end{split}
\end{equation}

If we found that $q_{\pi}(s,a)$ is greater than $v_{\pi}(s)$, then it means it is better off to select $a$ once in $s$ and thereafter follow $\pi$ all the time, and that new policy would in fact be a better one overall. The algorithm runs so on and so forth.

##### The policy improvement theorem
In general, Let $\pi$ and $\pi'$ be any pair of deterministic policies, such that, for all $s \in S$,

\begin{align}
q_{\pi}(s, \pi'(s)) \geq v_{\pi}(s)
\end{align}

Then, the policy $\pi'$ must be as good as, or better than, $\pi$, and thus it must obtain greater or equal expected return from all states $s \in S$:

\begin{align}
v_{\pi'}(s) \geq v_{\pi}(s)
\end{align}

In general, this can be extend to consider changes at all states and to all ossible actions.  
Suppose we have a new greedy policy $\pi'$ that select the action according to the max $q_{\pi}(s,a)$. 

\begin{equation}
\begin{split}
\pi'(s) & := \mathop{\arg\max}_{a} q_{\pi}(s,a) \\
& =  \mathop{\arg\max}_{a} \mathop{\mathbb{E}}[R_{t+1} + \gamma v_{\pi}(S_{t+1}) | S_t=s, A_t=a] \\
& = \mathop{\arg\max}_{a} \sum_{s',r}p(s',r|s,a)[r + \gamma v_{\pi}(s')]
\end{split}
\end{equation}

i.e. The greedy policy takes the action that looks best in the short term, according to the value $v_{\pi}$ of one step of lookahead

##### When to stop iteration?
Iteration stops when we found a policy, say $\pi'$, that is as good as, but not better than, the old policy $\pi$. That means, we have $v_\pi = v_{\pi'}$, then we can say $v_{\pi'}$ must be $v_*$, as it follows the Bellman optimality equation, and both $\pi$ and $\pi'$ must be optimal policies.

#### Algorithm

1. Initialization  
$V(s) \in \mathop{\mathbb{R}}$ and $\pi(s) \in A(s)$ arbitrarily for all $s \in S$

2. Policy Evaluation  
* Loop:  
$\Delta \leftarrow 0$
  * Loop for each $s \in S$:
     * $v \leftarrow V(s)$
     * $V(s) \leftarrow \sum_{s',r}p(s',r|s,\pi(s))[r+\gamma V(s')]$
     * $\Delta \leftarrow \mathop{\max}(\Delta, |v - V(s)|)$
* until $\Delta < \theta$

3. Policy Improvement  
* policy-stable $\leftarrow true$  
* For each $s \in S$:  
  * old-action $\leftarrow \pi(s)$
  * $\pi(s) \leftarrow \mathop{\arg\max}_{a} \sum_{s',r}p(s',r|s,a)[r + \gamma v_{\pi}(s')]$
  * If old-action $\neq \pi(s)$, then policy-stable $\leftarrow false$
* If policy-stable, then stop and return $V \approx v_*$ and $\pi \approx \pi_*$; else go to 2

In [0]:
def policy_improvement(policy, gamma=1.0, theta=1e-10):
  
  while True:
    
    policy_stable = True
    V = evaluate_policy(policy, gamma=gamma, theta=theta)

    for s in range(env.nS):

      q_list = []
      
      old_action = np.argmax(policy[s])

      # Base on V find the optimal action
      for a in range(env.nA):
        p, ns, r, _ = env.P[s][a][0]
        q_list.append(p * (r + gamma * V[ns]))

      new_action = np.argmax(q_list)
      
      # Evaluate if the policy is stable
      print("State: %s - old_action -> %s | new_action -> %s" % (s, old_action, new_action))
      if old_action != new_action:
        policy_stable = False
        
      policy[s] = np.eye(env.nA)[new_action]
      
    if policy_stable:
      return policy, V

In [77]:
# Iterate the whole process

random_policy = np.ones([env.nS, env.nA]) / env.nA

policy_improvement(random_policy)

Policy value evaluated.
State: 0 - old_action -> 0 | new_action -> 0
State: 1 - old_action -> 0 | new_action -> 3
State: 2 - old_action -> 0 | new_action -> 3
State: 3 - old_action -> 0 | new_action -> 3
State: 4 - old_action -> 0 | new_action -> 0
State: 5 - old_action -> 0 | new_action -> 0
State: 6 - old_action -> 0 | new_action -> 3
State: 7 - old_action -> 0 | new_action -> 2
State: 8 - old_action -> 0 | new_action -> 0
State: 9 - old_action -> 0 | new_action -> 0
State: 10 - old_action -> 0 | new_action -> 2
State: 11 - old_action -> 0 | new_action -> 2
State: 12 - old_action -> 0 | new_action -> 0
State: 13 - old_action -> 0 | new_action -> 1
State: 14 - old_action -> 0 | new_action -> 1
State: 15 - old_action -> 0 | new_action -> 0
Policy value evaluated.
State: 0 - old_action -> 0 | new_action -> 0
State: 1 - old_action -> 3 | new_action -> 3
State: 2 - old_action -> 3 | new_action -> 3
State: 3 - old_action -> 3 | new_action -> 2
State: 4 - old_action -> 0 | new_action -> 0
S

(array([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]]),
 array([ 0., -1., -2., -3., -1., -2., -3., -2., -2., -3., -2., -1., -3.,
        -2., -1.,  0.]))

In [59]:
policy_improvement(random_policy).reshape(4,4)

Policy value evaluated.


array([[0., 3., 3., 3.],
       [0., 0., 3., 2.],
       [0., 0., 2., 2.],
       [0., 1., 1., 0.]])